# Benchmark: Axisymmetric cooling and spreading

In [1]:
import rioxarray as rxr
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.insert(0,"/home/jovyan/shared/Libraries")
import victor
import cartopy.crs as ccrs
import subprocess
import os
import pandas as pd
import xarray as xr
import rasterio as rio
from rasterio.enums import Resampling

#### This benchmark is a temperature-dependent test on a viscous flow. 

In this experiment, a hot Newtonian fluid is extruded from a point source onto a horizontal plane and allowed to cool to the ambient air temperature. The propogation of such a flow has an analytical solution, which is plotted alongside the models.

We have the initial volume set in accordance with the [Garel et al., 2012](https://appliedvolc.biomedcentral.com/articles/10.1186/s13617-017-0061-x#ref-CR26) experiment.

We encourage changing the volume betweens runs to compare the physical or stochastic reactions at various scales.

In [ ]:
volume = 3.432e-6

#### The following cells should not be edited, unless you have extensive knowledge of the model.

The following cell sets up common parameters. Following this, the next 4 cells format and generate input files for each respective model, at parameters considered reasonable for this benchmark.

In [ ]:
easting = 0
northing = 0
dem = "Benchmark_Axisymmetric.asc"
raster = rxr.open_rasterio(dem)
raster = raster.sel({"band": 1})
xll, yll = float(raster.x.min()), float(raster.y.min())
coordinates = np.array([int(easting),int(northing)])
r = rio.open(dem)
res = r.res
ratio = volume/3.432e-6

In [ ]:
f = open("./Molasses/custom_molasses.conf","r+")
conf = f.readlines()
conf[2] = f"""MIN_RESIDUAL = {3e-8*ratio}\n"""
conf[3] = f"""MAX_RESIDUAL = {4e-5*ratio}\n"""
conf[4] = f"""MIN_TOTAL_VOLUME = {str(volume)}\n"""
conf[5] = f"""MAX_TOTAL_VOLUME = {str(volume)}\n"""
conf[6] = f"""MIN_PULSE_VOLUME = {float(volume)/100}\n"""
conf[7] = f"""MAX_PULSE_VOLUME = {float(volume)/100}\n"""
conf[10] = f"""DEM_FILE = {dem}\n"""
f.seek(0)
f.writelines(conf)
f.truncate()
f.close()

f=open("./Molasses/events.in","w")
f.write(f"""{easting},{northing}""")
f.close()

os.chdir("Molasses")
subprocess.run(f"""ln -s ../{dem} .""",shell=True)
subprocess.run("molasses custom_molasses.conf",shell=True)

victor.convert_molasses("molasses_axisymmetric")
os.chdir("..")

In [ ]:
g=open("./mr_lava_loba/input_data.py","r+")
inp = g.readlines()
inp[1] = f"""run_name = 'Axisymmetric'\n"""
inp[3] = f"""source = '{dem}'\n"""
inp[41] = f"""vent_flag = 0\n"""
inp[44] = f"""x_vent = [{int(easting)}]\n"""
inp[45] = f"""y_vent = [{int(northing)}]\n"""
inp[91]= f"""total_volume = {volume}\n"""
g.seek(0)
g.writelines(inp)
g.truncate()
g.close()

os.chdir("mr_lava_loba")
subprocess.run(f"""ln -s ../{dem} .""",shell=True)
subprocess.run("python mr_lava_loba.py",shell=True)
os.remove(dem)
os.chdir("..")

In [ ]:
radius = 2e-3/ratio
h=open("./IMEX/IMEX_LavaFlow.inp","r+")
inp = h.readlines()
inp[1] = 'RUN_NAME="Axisymmetric           ",\n'
inp[4] = f""" T_END=  156,\n"""
inp[5] = f""" DT_OUTPUT=  50,\n"""
inp[13] = f""" X0= {xll}D0     ,\n"""
inp[14] = f""" Y0= {yll}D0      ,\n"""
inp[17] = f""" CELL_SIZE = {res[0]}D0 ,\n"""
inp[96] = f"""  X_SOURCE=  {easting}.0D0 ,\n"""
inp[97] = f"""  Y_SOURCE=  {northing}.0D0 ,\n"""
inp[99] = f""" VEL_SOURCE= {float(volume)/(radius**2*np.pi*156)} \n"""
h.seek(0)
h.writelines(inp)
h.truncate()
h.close()

os.chdir("IMEX")
subprocess.run(f"""ln -s ../{dem} topography_dem.asc""",shell=True)
p = subprocess.Popen(['./IMEX_LavaFlow'], stdin=subprocess.PIPE, shell=True)
p.communicate(input=b'\n')
os.remove("topography_dem.asc")
os.chdir("..")

In [ ]:
scale = ratio

xres, yres = scale, scale

os.chdir("lava2d")

with rio.open("flat.tiff") as dataset:
    scale_factor_x = dataset.res[0]/xres
    scale_factor_y = dataset.res[1]/yres

    profile = dataset.profile.copy()
    data = dataset.read(
        out_shape=(
            dataset.count,
            161,
            161
        ),
        resampling=Resampling.nearest
    )

    transform = dataset.transform * dataset.transform.scale(
        (1 / scale_factor_x),
        (1 / scale_factor_y)
    )
    profile.update({"height": data.shape[-2],
                    "width": data.shape[-1],
                   "transform": transform})

with rio.open("Benchmark_Axisymmetric.tiff", "w", **profile) as dataset:
    dataset.write(data)

r = rxr.open_rasterio("Benchmark_Axisymmetric.tiff")
lonlat = r.rio.reproject("EPSG:4326")
xmin,ymin,xmax,ymax = lonlat.rio.bounds()
center = float(lonlat[:,80,80].x), float(lonlat[:,80,80].y)

k=open("./input.py","r+")
inp = k.readlines()
inp[4] = '    path_to_dem_file    = ("./Benchmark_Axisymmetric.tiff"),\n'
inp[5] = f"""    Lon_SRC             = {center[0]}, # source longitude\n"""
inp[6] = f"""    Lat_SRC             = {center[1]},    # source latitude\n"""
inp[7] = f"""    Lon_LowerLeft       = {xmin}, # bounding box: lower-left longitude\n"""
inp[8] = f"""    Lat_LowerLeft       = {ymin}, # bounding box: lower-left latitude\n"""
inp[9] = f"""    Lon_UpperRight      = {xmax}, # bounding box: upper-right longitude\n"""
inp[10] = f"""    Lat_UpperRight      = {ymax},   # bounding box: upper-right latitude\n"""
inp[67] = f"""    max_time_hr = {156/3600}\n"""

k.seek(0)
k.writelines(inp)
k.truncate()
k.close()

k=open("./example_vents/vent_01.txt","r+")
inp = k.readlines()
inp[1] = f'''0	{-5*scale}	{-5*scale}	{5*scale}	{5*scale}	{3*scale}	{2e-5*scale}\n'''

k.seek(0)
k.writelines(inp)
k.truncate()
k.close()

subprocess.run("python input.py", shell=True, stdout=subprocess.DEVNULL)

os.chdir("..")

#### The final two cells concern visualization.

The first provides a reference to the final timestamp of each simulation, though all outputs are avilable in their models' respective folder. The second and last cell plots each model side by side. We encourage users to edit this to their personal preference.

In [ ]:
r = rxr.open_rasterio("IMEX/Axisymmetric_0007.asc",masked=True).max()
r2 = rxr.open_rasterio("Molasses/molasses_axisymmetric.asc",masked=True).max()
r3 = rxr.open_rasterio("mr_lava_loba/Axisymmetric_000_thickness_masked.asc",masked=True).max()
r4 = xr.open_dataset("lava2d/outputs/out.nc",group="DATA/PHYSICS")
r4 = r4['lava_thickness_total'].max()/1000
maximum = np.max((r,r2,r3,r4))
idx = np.argmax((r,r2,r3,r4))

In [ ]:
fig, ((ax0, ax1), (ax2, ax3)) = plt.subplots(ncols=2, nrows=2,subplot_kw=dict(projection=ccrs.epsg(32628)), figsize = (12,9))

ticks = np.array([-.08, -.04, 0, .04, .08])

ax0.cla()
east, north = easting, northing

thickness0,maxval0 = victor.plot_benchmark(dem, "./Molasses/molasses_axisymmetric.asc", fig, ax0, coordinates, vmax=maximum, zoom=False)
ax0.set_title('MOLASSES', fontsize=16)

ax1.cla()
thickness1,maxval1 =victor.plot_benchmark(dem, "./mr_lava_loba/Axisymmetric_000_thickness_masked.asc", fig, ax1, coordinates, vmax=maximum, zoom=False,model="mrlavaloba")
ax1.set_title('MR LAVA LOBA', fontsize=16)
ax1.set_aspect("equal")

ax2.cla()
thickness2,maxval2 = victor.plot_benchmark(dem, "./IMEX/Axisymmetric_0007.asc", fig, ax2, coordinates, vmax=maximum, zoom=False)
ax2.set_title('IMEX', fontsize=16)
ax2.set_aspect("equal")

ax3.cla()
ds = xr.open_dataset("lava2d/outputs/out.nc",group="DATA/PHYSICS")
ds = ds.where(ds != 0)
ds = ds.where(ds > 1e-1)
ax3.imshow(ds['lava_thickness_total'], vmax=maximum*1000, cmap="Wistia")
ax3.set_xticks(np.linspace(0,161,5))
ax3.set_yticks(np.linspace(0,161,5))
ax3.set_xticklabels(ticks*1000)
ax3.set_yticklabels(np.array(list(reversed(ticks)))*1000)
ax3.set_title('LAVA2D', fontsize=16)
ax3.set_aspect("equal")

cbar_ax = fig.add_axes([.25, 0, 0.5, 0.05])
# fig.tight_layout()
thickness = [thickness0, thickness1, thickness2, (ds['lava_thickness_total']/1000)]
thickness = thickness[idx]
fig.colorbar(thickness, cax=cbar_ax, label="Flow Thickness (m)", orientation="horizontal")